# Resumo relatório final

## Modelo Memory Polynomial (MP)

O modelo MP é descrito pela seguinte equação: 

$$\tilde{out}_n = \sum^{P}_{p=1}\sum^{M}_{m=0}\tilde{coefs}_{mp}\tilde{in}_{n-m}|\tilde{in}_{n-m}|^{p-1}$$

Onde $\tilde{out}$ e $\tilde{in}$ são vetores de valores complexos.



## Cálculo dos coeficientes

O cálculo dos coeficientes é feito como descrito na equação a seguir:

$$ \tilde{coefs}_{mp} = \frac{\tilde{out}_n}  {\sum^{P}_{p=1}\sum^{M}_{m=0}\tilde{in}_{n-m}|\tilde{in}_{n-m}|^{p-1}} $$

Essa operação pode ser realizada utilizando a abordagem com matrizes complexas. 

In [1]:
import scipy.io as spio
import scipy as sp
import numpy as np
import matplotlib.pyplot as plt


mat = spio.loadmat('data_LDMOS.mat')
in_ext = mat['in_extraction']
out_ext = mat['out_extraction']
in_val = mat['in_validation']
out_val = mat['out_validation']

P = 5
M = 1
n = 2


def x_mp(entrada, M, P):
    modulo_entrada = np.absolute(entrada)
    X_MP = np.zeros((len(entrada),P*(M+1)),dtype=complex)
    for r in range(M+1,len(entrada)):
        for m in range(M+1):
            for p in range(1,P+1):
                c = ((m*P)-1)+p
                X_MP[r,c] = entrada[r-m,0]*((modulo_entrada[r-m,0])**(p-1))
    return X_MP

X_ext = x_mp(in_ext,M,P)
X_ext2 = X_ext[M+3:len(X_ext)-(M+3),:]
out_ext2 = out_ext[M+3:len(out_ext)-(M+3),:]
coefs = np.linalg.lstsq(X_ext2,out_ext2,rcond=-1)
coefs = coefs[0]
print(coefs)

[[ 1.30542312-0.0131647j ]
 [-1.76929598+0.06808608j]
 [ 3.90364644+0.66983932j]
 [-3.91249141-2.08893954j]
 [ 1.18305453+1.20344584j]
 [-0.14491305+0.01647957j]
 [ 0.50472549-0.18029606j]
 [-0.21917076+0.40431498j]
 [-0.45685679-0.44377121j]
 [ 0.34534536+0.18979976j]]


Os coeficientes como estão calculados são dispostos da seguinte forma: 

$$coef_{mp} = \begin{bmatrix} b_{01} \\ b_{02} \\ b_{03} \\ b_{11} \\ b_{12} \\ b_{13} \end{bmatrix} $$

Obs.: considerando M=1 e P=3


## Validação usando matrizes complexas:

Seguindo a equação:

$$\tilde{out}_n = \sum^{P}_{p=1}\sum^{M}_{m=0}\tilde{coefs}_{mp}\tilde{in}_{n-m}|\tilde{in}_{n-m}|^{p-1}$$

Para calcularmos a saída estimada para validação precisamos apenas multiplicar os coeficientes calculados anteriormente por $\sum^{P}_{p=1}\sum^{M}_{m=0}\tilde{in}_{n-m}|\tilde{in}_{n-m}|^{p-1}$


In [2]:
X_val = x_mp(in_val,M,P)
X_val2 = X_val[M+3:len(X_val)-(M+3),:]
out_calc_mat_cmplx = X_val2@coefs
print(out_calc_mat_cmplx)

[[0.23128482-0.40243892j]
 [0.16971226-0.44974441j]
 [0.09110992-0.46789906j]
 ...
 [0.12400208+0.11157424j]
 [0.11446275+0.14622802j]
 [0.10898914+0.17024472j]]


### NMSE 

A métrica utilizada comparar o desempenho do modelo em cada abordagem será o NMSE, que é descrito pela seguinte equação:

$$ NMSE = 10log_{10}\frac{\sum^{N}_{n=1}|e_n|^2}{\sum^{N}_{n=1}|out_n|^2}$$

onde $out_n$ é o valor da amostra do sinal de saída no instante n, $e_n$ é o valor calculado
do erro entre as amostras do sinal de saída real e do sinal de saída simulado pelo modelo,
ou seja, $out_n − outcalc_n$ e N representa a quantidade total de amostras disponíveis.

In [10]:
def NMSE(previsao,validacao):
    erro = validacao - previsao
    erro = np.sum(np.absolute(erro)**2)
    val = np.sum(np.absolute(validacao)**2)
    res = 10*np.log10(erro/val)
    return res
out_val2 = out_val[M+3:len(X_val)-(M+3),:]
NMSE(out_calc_mat_cmplx,out_val2)

-38.07449082171138

## Validação usando LUTS:

In [11]:
def in_abs_lut(entrada, M, P):
    modulo_entrada = np.absolute(entrada)
    in_abs = np.zeros((len(entrada),P*(M+1)),dtype=complex)
    for r in range(M+1,len(entrada)):
        for m in range(M+1):
            for p in range(1,P+1):
                c = ((m*P)-1)+p
                in_abs[r,c] = (modulo_entrada[r-m,0])**(p-1)
    return in_abs


def lut(in_abs,M,P,n):
    xpol2 = in_abs[:,0:P-1]@coefs[0:P-1]
    for i in range(1,M+1):
        xpol = in_abs[:,i*P:((i*P)*(i+1))-1]@coefs[i*P:((i*P)*(i+1))-1]
        xpol2 = np.hstack((xpol2,xpol))

    xpol2 = xpol2[M+3:len(X_val)-(M+3),:]

    Q = 2**n
    z=np.linspace(0,1,Q)

    z = [int(round(a*(len(xpol2)-1))) for a in z]  
    
    lut = np.zeros((Q,M+1),dtype=np.complex_)
    for c in range(0,M+1):
        for r in range(Q):
            lut[r,c] = xpol2[z[r],c]
    return lut


in_lut = in_abs_lut(in_val,M,P)
lut = lut(in_lut, M, P, n)
print('LUT: ',lut)

LUT:  [[ 0.93276386-0.0473117j  -0.00507413-0.02117153j]
 [ 1.01531844+0.01003029j -0.02789541-0.01267545j]
 [ 1.01313211+0.00945178j -0.04075718-0.01002357j]
 [ 1.07789517+0.0103456j  -0.0636315 -0.00550209j]]


In [12]:
from scipy import interpolate


def interpolacao(entrada,lut,M,n):
    Q = 2**n
    z = np.linspace(0,1,Q)
 
    abs_in = np.absolute(entrada)

    inter = np.interp(abs_in,z,lut[:,0])
    for c in range(1,M+1):
        inter2 = np.interp(abs_in,z,lut[:,c])
        inter = np.hstack((inter,inter2))

    return inter

inter = interpolacao(in_val,lut,M,n)
inter = inter.sum(axis=1)
inter = inter.reshape(len(inter),1)

out_lut = inter*in_val

erro_lut = out_val - out_lut
erro_lut = np.sum(np.absolute(erro_lut)**2)
out_ref_lut = np.sum(np.absolute(out_val2)**2)
NMSE_lut = 10*np.log10(erro_lut/out_ref_lut)
print(NMSE_lut)

-25.37310917380284


## Validação usando matrizes reais:

Usando a equação:


$\begin{bmatrix} a + jb & c + jd \end{bmatrix} * \begin{bmatrix} e + jf \\ g + jh \end{bmatrix} = \begin{bmatrix} a & b & c & d \end{bmatrix} * \begin{bmatrix} e & f \\ -f & e \\ g & h \\ -h & g \end{bmatrix}$

Na etapa de multiplicação pelos coeficientes.

In [13]:
def in_abs_real(in_val,P):
    res = np.zeros((len(in_val),P))
    for c in range(1,P+1,1):
        for r in range(len(in_val)):
            res[r,c-1] = (np.abs(in_val[r]))**(c-1)
    return res


def x_mp_real(in_val,M):
    res0 = np.zeros((len(in_val),M+1),dtype=complex)
    for c in range(M+1):
        for r in range(len(in_val)-c):
            res0[r+c,c] = in_val[r,0]

    res = np.zeros((len(in_val),2*(M+1)))   
    c2=0   
    for c in range(0,2*(M+1),2):
            res[:,c] = np.real(res0[:,c2])
            res[:,c+1] = np.imag(res0[:,c2])
            c2+=1
    return res


in_real = in_abs_real(in_val,P)
x_real = x_mp_real(in_val,M)

In [14]:
def mult1(in_real,x_real,M,P):
    res = np.zeros((len(in_real),2*P*(M+1)))
    zero = np.zeros((1,in_real.shape[1]))
    c=-2
    for c1 in range(0,2*(M+1),2):
        if c1 != 0:
            in_real = np.append(zero,in_real,axis=0)
            in_real = in_real[0:in_real.shape[0]-1,:]
        for c2 in range(P):
            c+=2
            res[:,c] = x_real[:,c1]*in_real[:,c2] 
            res[:,c+1] = x_real[:,c1+1]*in_real[:,c2]
    res = res[0:len(res)-M,:]
    return res


def coef_real(coef):
    res0 = np.zeros((len(coef),2))
    for r in range(len(coef)):
        res0[r,0] = np.real(coef[r])
        res0[r,1] = np.imag(coef[r])
    res = np.zeros((2*len(coef),2))
    r2=0
    for r in range(len(res0)):
        res[r2,0]=res0[r,0]
        res[r2,1]=res0[r,1]
        res[r2+1,0]=-res0[r,1]
        res[r2+1,1]=res0[r,0]
        r2+=2
    return res

mult1_re = mult1(in_real,x_real,M,P)
coef_re = coef_real(coefs)
mult2_re = mult1_re@coef_re
out_mat_re = mult2_re[:,0]+(mult2_re[:,1]*1j)
out_mat_re = out_mat_re.reshape(len(out_mat_re),1)
print(out_mat_re)

[[0.21868565+0.01824158j]
 [0.25150858-0.11443982j]
 [0.27300396-0.22676049j]
 ...
 [0.10560923+0.17762753j]
 [0.10223906+0.16304819j]
 [0.09719938+0.12272305j]]


In [15]:
out_val_re = out_val[0:len(out_val)-M,:]
NMSE(out_mat_re,out_val_re)

-38.07242351982885